👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

In [16]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q duckduckgo_search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.2 MB/s eta 0:00:0000:0100:01


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
#We first make a tool to get cargo plane transfer time
import math
from typing import Optional , Tuple
from smolagents import tool

In [5]:
@tool
def calculate_cargo_travel_time(
    origin_coords:Tuple[float,float],
    destination_coords:Tuple[float,float],
    cruising_speed_kmh:Optional[float]=750.0 #Average speed for cargo planes
    ) -> float:

    """ Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.
    
    Args:
        origin_coords: Tuple of (latitude,longitude) for starting point
        destination_coords: Tuple of (latitude and longitude) for end point
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750.0 km/h for typical cargo planes)
    
    Returns:
        float:  The estimated travel time in hours

    Example:
        >>> #Ikeja (6.6018° N, 3.3515° E) TO Apapa (6.4483° N ,3.3675° E)
        >>> result = calculate_cargo_travel_time((6.6018, 3.3515), (6.4483, 3.3675))
    """
    def to_radians(degrees:float) -> float:
        return degrees * (math.pi/180)

    #Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    #Earth's radius in km
    Earth_radius_km = 6371.0

    #Calculate great_circle distance using haversine formula
    dlon= lon2 - lon1
    dlat= lat2 - lat1

    a = (
        math.sin(dlat/2) **2 
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2 
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = Earth_radius_km * c

    #Add 10% to account for routes that are not direct and air-traffic controls
    actual_distance = distance * 1.1

    #Calculate flight time
    #Add 1 hour for take off and landing procedure
    flight_time = (actual_distance/cruising_speed_kmh) + 1.0
    #format the results
    return round(flight_time,2)
    

In [8]:
print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))) #Chicago to Sydney

22.82


For the model provider, we use Together AI, one of the new inference providers on the Hub!

Regarding the GoogleSearchTool: this requires either having setup env variable SERPAPI_API_KEY and passing provider="serpapi" or having SERPER_API_KEY and passing provider=serper.

If you don't have any Serp API provider setup, you can use DuckDuckGoSearchTool but beware that it has a rate limit.

In [12]:
import os
from PIL import Image
from smolagents import CodeAgent ,GoogleSearchTool, HfApiModel, VisitWebpageTool,DuckDuckGoSearchTool

model = HfApiModel(model_id = "Qwen/Qwen2.5-Coder-32B-Instruct", provider = "together")

We can start with creating a baseline, simple agent to give us a simple report.

In [11]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo
plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [18]:
agent = CodeAgent(model = model, tools = [DuckDuckGoSearchTool(),VisitWebpageTool(),calculate_cargo_travel_time],
                 additional_authorized_imports = ["pandas"],max_steps = 20)

In [19]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo                        │
│ plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.           │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search("Batman filming locations")                                                        
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
Matt Reeves' The Batman is set in Gotham City, and here are the real-life filming locations around the world used 
to bring the fictional city to life. The Batman was a worldwide production. Screen Rant

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Where Was The Batman Filmed? All the Iconic Locations Explored - 
OtakuKart](https://otakukart.com/where-was-the-batman-filmed-all-the-iconic-locations-explored/)
The Batman, directed by Matt Reeves, is a gritty, noir-infused take on Gotham City, featuring Robert Pattinson as 
the latest incarnation of the Dark Knight.While the film's setting feels uniquely atmospheric, the shooting 
locations are scattered across the United Kingdom and Chicago, blending real-world settings with digital 
enhancements to bring Gotham to life.

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The filming of The Batman film started in January 2020 under the working title Vengeance and wrapped on March 21, 
2021. The second unit filming took place from 17 September 2020 to 13 March 2021. In the below article, we will see
the different locations where The Batman film was filmed. Here is a guide to all the locations of The Batman 
Filming ...

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
Filming on real locations helped create this new version of Gotham. In the past, movies with Batman have shown 
Gotham in different ways. Gotham is usually imagined as a part of New Jersey in the comics. ... It was established 
in 1863. This location was also used to film Batman and Catwoman's motorcycle chase. Anfield Cemetery, Liverpool 
...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts. Some portions were also filmed in Illinois, particularly 
in Chicago. Principal photography for the movie commenced on January 27, 2020, but was abruptly halted on March 14 
...

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
Where Was The Batman Filmed? The Batman movie was filmed in various locations around

[Step 1: Duration 3.79 seconds| Input tokens: 2,276 | Output tokens: 164]

...

[Step 4: Duration 18.64 seconds| Input tokens: 15,463 | Output tokens: 2,067]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Supercar factories coordinates                                                                                 
  supercar_coords = [                                                                                              
      (44.5964, 10.8856),  # Ferrari Maranello                                                                     
      (44.6975, 11.1948),  # Lamborghini Sant'Agata Bolognese                                                      
      (44.7500, 10.9667),  # Pagani San Cesario sul Panaro                                                         
      (51.3778, -0.5714),  # McLaren Woking                                                                        
      (58.4110, 15.6215)   # Koenigsegg Gemeneboden                                                                
  ]                                                                                                                
                                                                                                                   
  # Calculate travel times for supercar factories                                                                  
  supercar_travel_times = []                                                                                       
  for coord in supercar_coords:                                                                                    
      distance = calculate_distance(coord, gotham_coords)                                                          
      travel_time = calculate_travel_time(distance)                                                                
      supercar_travel_times.append(travel_time)                                                                    
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame({                                                                                     
      'Location': ['Ferrari Maranello', 'Lamborghini Sant\'Agata Bolognese', 'Pagani San Cesario sul Panaro',      
  'McLaren Woking', 'Koenigsegg Gemeneboden'],                                                                     
      'Latitude': [coord[0] for coord in supercar_coords],                                                         
      'Longitude': [coord[1] for coord in supercar_coords],                                                        
      'Travel Time (hours)': supercar_travel_times                                                                 
  })                                                                                                               
                                                                                                                   
  # Print the DataFrames                                                                                           
  print("Batman Filming Locations:")                                                                               
  print(batman_df)                                                                                                 
  print("\nSupercar Factories:")                                                                                   
  print(supercar_df)                                                                                               
                                                                                                                   
  # Return the DataFrames as the final answer                                                                      
  final_answer((batman_df, supercar_df))                                                                           
 ────────────────────────────────────────────────────────

Execution logs:
Batman Filming Locations:
    Location   Latitude  Longitude  Travel Time (hours)
0     London  51.511531  -0.112160             7.428140
1    Glasgow  55.862116  -4.230996             6.910755
2  Liverpool  53.408366  -2.991572             7.098441
3    Chicago  41.878113 -87.629799             1.525722

Supercar Factories:
                           Location  Latitude  Longitude  Travel Time (hours)
0                 Ferrari Maranello   44.5964    10.8856             8.836413
1  Lamborghini Sant'Agata Bolognese   44.6975    11.1948             8.857686
2     Pagani San Cesario sul Panaro   44.7500    10.9667             8.832893
3                    McLaren Woking   51.3778    -0.5714             7.394046
4            Koenigsegg Gemeneboden   58.4110    15.6215             8.312714

Out - Final answer: (    Location   Latitude  Longitude  Travel Time (hours)
0     London  51.511531  -0.112160             7.428140
1    Glasgow  55.862116  -4.230996             6.910755
2  Liverpool  53.408366  -2.991572             7.098441
3    Chicago  41.878113 -87.629799             1.525722,                            Location  Latitude  Longitude  
Travel Time (hours)
0                 Ferrari Maranello   44.5964    10.8856             8.836413
1  Lamborghini Sant'Agata Bolognese   44.6975    11.1948             8.857686
2     Pagani San Cesario sul Panaro   44.7500    10.9667             8.832893
3                    McLaren Woking   51.3778    -0.5714             7.394046
4            Koenigsegg Gemeneboden   58.4110    15.6215             8.312714)

[Step 5: Duration 20.71 seconds| Input tokens: 22,313 | Output tokens: 3,567]

In [20]:
result

(    Location   Latitude  Longitude  Travel Time (hours)
 0     London  51.511531  -0.112160             7.428140
 1    Glasgow  55.862116  -4.230996             6.910755
 2  Liverpool  53.408366  -2.991572             7.098441
 3    Chicago  41.878113 -87.629799             1.525722,
                            Location  Latitude  Longitude  Travel Time (hours)
 0                 Ferrari Maranello   44.5964    10.8856             8.836413
 1  Lamborghini Sant'Agata Bolognese   44.6975    11.1948             8.857686
 2     Pagani San Cesario sul Panaro   44.7500    10.9667             8.832893
 3                    McLaren Woking   51.3778    -0.5714             7.394046
 4            Koenigsegg Gemeneboden   58.4110    15.6215             8.312714)

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [21]:
agent.planning_interval = 4

detailed_report = agent.run(f"""You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}""")
print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo                        │
│ plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.           │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We are in Gotham and need to calculate travel time to various locations via cargo plane.
- We need to find Batman filming locations worldwide.
- We need to find supercar factories with the same cargo plane transfer time as the Batman filming locations.

### 1.2. Facts to look up
- Coordinates of all known Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations coordinates"
- Coordinates of supercar factories worldwide.
  - Source: Web search for "supercar factories coordinates"
- Cruising speed of a typical cargo plane.
  - Source: Web search for "cargo plane cruising speed"

### 1.3. Facts to derive
- Travel time from each Batman filming location to Gotham via cargo plane.
  - Derivation: Use the `calculate_cargo_travel_time` function with Gotham coordinates and each filming location's 
coordinates.
- Travel time from each supercar factory to Gotham via cargo plane.
  - Derivation: Use the `calculate_cargo_travel_time` function with Gotham coordinates and each supercar factory's 
coordinates.

## 2. Plan
1. Perform a web search to find the coordinates of all known Batman filming locations worldwide.
2. Visit the source URLs of the Batman filming locations to confirm the coordinates.
3. Perform a web search to find the coordinates of supercar factories worldwide.
4. Visit the source URLs of the supercar factories to confirm the coordinates.
5. Perform a web search to find the cruising speed of a typical cargo plane.
6. Visit the source URL of the cargo plane cruising speed to confirm the value.
7. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
8. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
9. Create a pandas dataframe containing the Batman filming locations, their coordinates, and the calculated travel 
time to Gotham.
10. Create a pandas dataframe containing the supercar factories, their coordinates, and the calculated travel time 
to Gotham.
11. Filter the supercar factories dataframe to include only those with the same travel time as the Batman filming 
locations.
12. Combine the Batman filming locations dataframe and the filtered supercar factories dataframe into a single 
dataframe.
13. Return the combined dataframe as the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Batman filming locations coordinates                                                                
  batman_filming_locations_search = web_search("Batman filming locations coordinates")                             
  print("Batman filming locations search results:", batman_filming_locations_search)                               
                                                                                                                   
  # Search for supercar factories coordinates                                                                      
  supercar_factories_search = web_search("supercar factories coordinates")                                         
  print("Supercar factories search results:", supercar_factories_search)                                           
                                                                                                                   
  # Search for cargo plane cruising speed                                                                          
  cargo_plane_cruising_speed_search = web_search("cargo plane cruising speed")                                     
  print("Cargo plane cruising speed search results:", cargo_plane_cruising_speed_search)                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations search results: ## Search Results

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
Matt Reeves' The Batman is set in Gotham City, and here are the real-life filming locations around the world used 
to bring the fictional city to life. The Batman was a worldwide production. Screen Rant

[The Batman (2022) Locations - Movies Locations - 
LatLong](https://www.latlong.net/location/the-batman-2022-locations-427)
The Batman (2022) locations list with spot information and geo coordinates. View the locations of The Batman (2022)
on map. ... Based on the DC Comics character Batman, the film is a reboot of the Batman film franchise and was 
produced by DC Films and distributed by Warner Bros. Pictures. Starring Robert Pattinson, Zoë Kravitz, Paul Dano 
...

[Where was the movie The Batman filmed: | Filming locations, countries 
...](https://wherefilmed.org/en/the-batman-2022/)
Royal Liver Building on the map. Scene was shot The Royal Liver Building is one of the most recognisable landmarks 
in the city of Liverpool with its two fabled Liver Birds which watch over the city and the sea. It is located at 
the Pier Head and along with the neighbouring Cunard Building and Port of Liverpool Building is one of Liverpool's 
Three Graces, which line the city's waterfront.

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The filming of The Batman film started in January 2020 under the working title Vengeance and wrapped on March 21, 
2021. The second unit filming took place from 17 September 2020 to 13 March 2021. In the below article, we will see
the different locations where The Batman film was filmed. Here is a guide to all the locations of The Batman 
Filming ...

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
When filming on location, the actors and crew primarily stayed in the United Kingdom to capture Gotham's key 
locations. The Chicago portion of the shoot was primarily done with a second unit crew for exterior shots. Gotham 
was shown at a variety of locations in the United Kingdom. Here are the real-life filming sites for 'The Batman.'

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts. Some portions were also filmed in Illinois, particularly 
in Chicago. Principal photography for the movie commenced on January 27, 2020, but was abruptly halted on March 14 
...

[The Batman (2021) Film Locations | Global Film 
Locations](https://globalfilmlocations.net/2020/08/23/the-batman-2021-film-locations/)
Filming locations so far 2021 The Batman movie starring Robert Pattinson. Locations: 2 Temple Place, Temple, 


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Manually extracted Batman filming locations coordinates                                                        
  batman_filming_locations = {                                                                                     
      "Royal Liver Building, Liverpool": (53.4084, -2.9916),                                                       
      "Glasgow Necropolis, Glasgow": (55.8621, -4.2310),                                                           
      "2 Temple Place, London": (51.5115, -0.1122),                                                                
      "Holborn Tram Tunnel, London": (51.5200, -0.1050),                                                           
      "Chicago": (41.8781, -87.6298)                                                                               
  }                                                                                                                
                                                                                                                   
  # Manually extracted supercar factories coordinates                                                              
  supercar_factories = {                                                                                           
      "McLaren, Woking, England": (51.3000, -0.5999),                                                              
      "Porsche, Leipzig, Germany": (51.3397, 12.3731),                                                             
      "Ferrari, Maranello, Italy": (43.8343, 10.9921),                                                             
      "Aston Martin, Gaydon, England": (52.3774, -1.3151),                                                         
      "Acura, Marysville, USA": (39.7817, -121.8291)                                                               
  }                                                                                                                
                                                                                                                   
  # Extract cargo plane cruising speed from search results                                                         
  import re                                                                                                        
                                                                                                                   
  cruising_speed_pattern = re.compile(r"(\d+)\s*(km/h|mph)")                                                       
  cruising_speed_matches = cruising_speed_pattern.findall(cargo_plane_cruising_speed_search)                       
                                                                                                                   
  # Convert all speeds to km/h                                                                                     
  cruising_speeds_kmh = []                                                                                         
  for speed, unit in cruising_speed_matches:                                                                       
      if unit == "mph":                                                                                            
          speed_kmh = float(speed) * 1.60934                                                                       
      else:                                                                                                        
          speed_kmh = float(speed)                                                                                 
      cruising_speeds_kmh.append(speed_kmh)                                                                        
                                                                                                                   
  # Calculate the average cruising speed                 

Execution logs:
Average cargo plane cruising speed (km/h): 810.34679

Out - Final answer:                           Location          Coordinates  Travel Time (hours)
0  Royal Liver Building, Liverpool   (53.4084, -2.9916)                 8.23
1      Glasgow Necropolis, Glasgow    (55.8621, -4.231)                 8.04
2           2 Temple Place, London   (51.5115, -0.1122)                 8.56
3      Holborn Tram Tunnel, London      (51.52, -0.105)                 8.56
4                          Chicago  (41.8781, -87.6298)                 2.55

[Step 4: Duration 23.47 seconds| Input tokens: 30,360 | Output tokens: 3,146]

                          Location          Coordinates  Travel Time (hours)
0  Royal Liver Building, Liverpool   (53.4084, -2.9916)                 8.23
1      Glasgow Necropolis, Glasgow    (55.8621, -4.231)                 8.04
2           2 Temple Place, London   (51.5115, -0.1122)                 8.56
3      Holborn Tram Tunnel, London      (51.52, -0.105)                 8.56
4                          Chicago  (41.8781, -87.6298)                 2.55


In [22]:
detailed_report

,Location,Coordinates,Travel Time (hours)
0,"Royal Liver Building, Liverpool","(53.4084, -2.9916)",8.23
1,"Glasgow Necropolis, Glasgow","(55.8621, -4.231)",8.04
2,"2 Temple Place, London","(51.5115, -0.1122)",8.56
3,"Holborn Tram Tunnel, London","(51.52, -0.105)",8.56
4,Chicago,"(41.8781, -87.6298)",2.55


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs.

➡️ We need to improve the structure of our system.

✌️ Splitting the task between two agents
Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:

Each agent is more focused on its core task, thus more performant
Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.
Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including matplotlib, and geopandas + shapely for spatial plotting.

In [24]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [25]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY")

In [28]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [29]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude,longitude) for starting     │
│   │                             │ using great-circle distance.          │ point                                 │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude and longitude) for end   │
│   │                             │                                       │ point                                 │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750.0 km/h for typical   │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude,longitude) for starting │
            │                             │ Earth using great-circle          │ point                             │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │               

In [30]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here 
(we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least
6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with 
a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
<think>
Okay, let's tackle this task step by step. The user wants Batman filming locations, calculate cargo plane times 
from Gotham (40.7128° N, 74.0060° W), find supercar factories with the same travel time, and create a map with at 
least 6 points. 

First, I need to figure out the facts. The given facts are the coordinates for Gotham, which is actually New York 
City. The example code uses plotly to create a map, so that's the tool to use. 

For the facts to look up, I need a list of all Batman filming locations. The web agent can search for that. Each 
location's coordinates are necessary for calculating travel time. Then, for supercar factories, again, the web 
agent can find those, but I need to ensure they have the same travel time as some of the Batman locations. Wait, 
the user wants at least 6 points in total, so maybe 3 Batman and 3 supercar factories, but the exact number depends
on the data.

Calculating travel time for each Batman location using the given function. Then, for each supercar factory, 
calculate their travel time from Gotham and check if it matches any of the Batman locations' times. But how to 
determine "same" time? Maybe within a certain range, like ±1 hour? Or group them into similar times. The problem 
says "same cargo plane transfer time," so exact matches might be needed, but realistically, maybe cluster them.

Once all locations and their times are gathered, compile them into a dataframe with lat, lon, name, and travel 
time. Then use plotly to create the scatter map with color based on travel time. Save the image as saved_map.png.

Possible issues: Some Batman locations might not have coordinates readily available. The web agent needs to find 
accurate ones. Similarly, supercar factories' locations might be spread out, so their travel times might vary. Need
to find enough factories that match the times from Batman locations. If not enough, maybe adjust the criteria.

Plan steps: First, get Batman locations and their coordinates. Then compute each's travel time. Then find supercar 
factories, compute their times, and select those matching existing times. If not enough, maybe find more factories 
or adjust the time matching. Then combine all points, plot, and save.
</think>

## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W (equivalent to New York City)
- Example plotting code using Plotly
- Cargo plane speed defaults to 750 km/h (implied by `calculate_cargo_travel_time` tool)
- Required output: saved_map.png with ≥6 points (Batman locations + supercar factories)

### 1.2. Facts to look up
1. **Batman filming locations** (via web_agent):
   - List of all movies/TV shows featuring Batman
   - Specific filming locations for each production
   - Geographic coordinates for each location
   Source: IMDb/Wikipedia/film tourism websites

2. **Supercar factory locations** (via web_agent):
   - List of manufacturers producing supercars (e.g. Ferrari, Lamborghini, Bugatti)
   - Headquarters/factory locations
   - Geographic coordinates for each factory
   Source: Manufacturer websites/automotive industry databases

### 1.3. Facts to derive
1. Travel times for Batman locations:
   - Use `calculate_cargo_travel_time` with Gotham coordinates vs each filming location

2. Travel times for supercar factories:
   - Calculate for each factory using same function

3. Time-based matching:
   - Identify factories with travel times matching any Batman location's time (±tolerance)
   - Ensure total combined unique locations ≥6

4. Color mapping scheme:
   - Normalized travel time scale for consistent color coding
   - Association between time values and Magma colormap

## 2. Plan

1. Retrieve comprehensive list of Batman filming locations using web_agent
2. Extract

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Okay, let's tackle this step by step. First, I need to find all the Batman filming locations. I'll use the         
web_agent tool to search for that. Then, I have to extract their coordinates. Once I have those, I can calculate   
the travel time from Gotham (which is NYC's coordinates) using the calculate_cargo_travel_time function.           
                                                                                                                   
Next, I need to find supercar factories. Again, use web_agent to get a list of them and their locations. Calculate 
their travel times too. Then, match factories to Batman locations where the travel times are the same. Need at     
least 6 points total, so maybe some Batman locations and some factories.                                           
                                                                                                                   
Once I have all the data, I'll create a DataFrame with the locations, their coordinates, and travel times. Then use
Plotly to plot them on a map with colors based on travel time. Save the image and return it as the final answer.   
                                                                                                                   
Wait, but how do I handle the web_agent responses? They might return text, so I need to parse that to get          
coordinates. Maybe the web_agent can find structured data, but if not, I might need to manually input coordinates  
if the response includes [38;2;230;237;

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_task = "Find all Batman movie filming locations worldwide with their latitude/longitude coordinates in    
  decimal format. Return as list of tuples: (location_name, lat, lon)"                                             
  batman_raw = web_agent(batman_task)                                                                              
  print("Batman locations raw data:\n", batman_raw)                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Check check_reasoning_and_plot failed with error: The api_key client option must be set either by passing api_key 
to the client or by setting the OPENAI_API_KEY environment variable

[Step 15: Duration 9.69 seconds| Input tokens: 135,995 | Output tokens: 21,575]

Reached max steps.

[Step 16: Duration 26.44 seconds| Input tokens: 151,819 | Output tokens: 22,609]

'<think>\n\nOkay, let\'s tackle this problem step by step. The user wants a map showing Batman filming locations and supercar factories with similar cargo plane transfer times from Gotham (NYC coordinates). \n\nFirst, I need to gather Batman filming locations. The previous attempts showed that using the web_agent to search for these locations worked, but there were issues with parsing the data. The agent returned a list of locations with coordinates. However, when trying to access them, there were errors because the data wasn\'t properly converted from a string to a dictionary. So, I need to make sure that the data is correctly parsed. Since using eval or ast wasn\'t allowed, using regex to extract the tuples seems like the right approach.\n\nNext, calculate the travel times from Gotham to each Batman location using the calculate_cargo_travel_time function. This function likely computes the great-circle distance and divides by the cargo plane speed (assumed 750 km/h). \n\nThen, find su

In [33]:
manager_agent.python_executor.state["fig"]

{'data': [{'type': 'scattermapbox',
   'lat': [53.408366,
    51.507351,
    51.653811,
    52.132633,
    55.864237,
    55.804719,
    41.878113,
    51.508014,
    40.712776,
    37.774929,
    52.520007,
    41.385064,
    43.65107,
    53.349805,
    47.497912,
    44.5323,
    44.6586,
    48.5318,
    51.3334,
    44.9524,
    57.4907],
   'lon': [-2.991572,
    -0.127758,
    -0.406253,
    -0.431811,
    -4.251806,
    -3.782781,
    -87.629799,
    -0.128035,
    -74.005974,
    -122.419418,
    13.404954,
    2.173403,
    -79.347015,
    -6.26031,
    19.040235,
    10.8641,
    11.1277,
    7.5124,
    -0.7425,
    9.9936,
    12.6735],
   'mode': 'markers',
   'marker': {'size': 9,
    'color': [8.81,
     9.17,
     9.14,
     9.11,
     8.6,
     8.64,
     2.68,
     9.17,
     1.0,
     7.06,
     10.36,
     10.04,
     1.8,
     8.5,
     11.28,
     10.72,
     10.74,
     10.09,
     9.12,
     10.6,
     9.97],
    'colorscale': 'Magma',
    'showscale': True},
 